<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: 
* Reward function: 
* State: 
* Environment: 

<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages

In [1]:
!pip install -r requirements.txt --user

  Cloning git://github.com/dvdpedros/pyfolio (to revision fix-extreme-drawdown) to /tmp/pip-req-build-u60w2b3o
  Running command git clone -q git://github.com/dvdpedros/pyfolio /tmp/pip-req-build-u60w2b3o
  Running command git checkout -b fix-extreme-drawdown --track origin/fix-extreme-drawdown
  Switched to a new branch 'fix-extreme-drawdown'
  Branch 'fix-extreme-drawdown' set up to track remote branch 'fix-extreme-drawdown' from 'origin'.


<a id='1.2'></a>
## 2.2. Import Packages

In [2]:
%load_ext autoreload
%autoreload 
import pandas as pd
from config import config
from dataset.download_dataset.cryptodownloader_binance import CryptoDownloader_binance
from preprocessing.preprocessors import FeatureEngineer
from preprocessing.data import data_split
from env.env_custom import CustomTradingEnv
from model.models import DRLAgent
from trade.backtest import BackTest
import warnings
warnings.filterwarnings("ignore")


/home/joan/.local/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.3'></a>
## 2.3 Create Folders

In [3]:
import os
download_data = False
if not os.path.exists(config.DATA_SAVE_DIR):
    os.makedirs(config.DATA_SAVE_DIR)
    download_data = True
if not os.path.exists(config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists(config.TENSORBOARD_LOG_DIR):
    os.makedirs(config.TENSORBOARD_LOG_DIR)
if not os.path.exists(config.RESULTS_DIR):
    os.makedirs(config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data

In [32]:
data_downloader = CryptoDownloader_binance(config.START_DATE, config.END_DATE, config.ACTUAL_TICKERS, config.DATA_SAVE_DIR, config.DATA_GRANULARITY)
if download_data:    
    data_downloader.download_data()
df = data_downloader.load()

In [33]:
df

,date,open,high,low,close,volume,tic
0,2020-01-01 00:00:00,7195.24,7196.25,7175.46,7177.02,511.814901,btc
1,2020-01-01 01:00:00,7176.47,7230.00,7175.71,7216.27,883.052603,btc
2,2020-01-01 02:00:00,7215.52,7244.87,7211.41,7242.85,655.156809,btc
3,2020-01-01 03:00:00,7242.66,7245.00,7220.00,7225.01,783.724867,btc
4,2020-01-01 04:00:00,7225.00,7230.00,7215.03,7217.27,467.812578,btc
...,...,...,...,...,...,...,...
105019,2021-06-30 19:00:00,140.83,141.76,140.33,141.55,21511.253020,ltc
105020,2021-06-30 20:00:00,141.63,142.91,140.70,142.16,32348.880040,ltc
105021,2021-06-30 21:00:00,142.16,143.86,142.14,143.25,23128.015840,ltc
105022,2021-06-30 22:00:00,143.25,144.41,142.39,143.50,22183.430460,ltc


# Part 4: Preprocess Data

In [5]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = True)

df = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added user defined features


In [6]:
df.to_pickle(config.DATA_SAVE_DIR+"/preprocess_df.pkl")

In [47]:
#df_preprocess = fe.preprocess_data(df)
data_split(df_preprocess,"2021-06-01", "2021-07-01")


,date,open,high,low,close,volume,tic,psar,ui,atr,...,cov_4_4,cov_4_5,cov_4_6,cov_4_7,cov_5_5,cov_5_6,cov_5_7,cov_6_6,cov_6_7,cov_7_7
0,2021-06-01 00:00:00,1.73760,1.77960,1.72430,1.77190,3.406147e+07,ada,0.0,0.809046,0.037506,...,0.000188,0.000189,0.000179,0.000160,0.000302,0.000215,0.000205,0.000273,0.000209,0.000497
0,2021-06-01 00:00:00,353.37000,364.40000,351.08000,363.21000,2.212829e+05,bnb,0.0,1.068685,7.945713,...,0.000188,0.000189,0.000179,0.000160,0.000302,0.000215,0.000205,0.000273,0.000209,0.000497
0,2021-06-01 00:00:00,37253.82000,37894.81000,37100.00000,37798.31000,5.234073e+03,btc,0.0,0.590542,571.263400,...,0.000188,0.000189,0.000179,0.000160,0.000302,0.000215,0.000205,0.000273,0.000209,0.000497
0,2021-06-01 00:00:00,0.32558,0.32891,0.32400,0.32862,7.426512e+07,doge,0.0,0.742394,0.005788,...,0.000188,0.000189,0.000179,0.000160,0.000302,0.000215,0.000205,0.000273,0.000209,0.000497
0,2021-06-01 00:00:00,2706.15000,2740.00000,2690.00000,2730.67000,5.121758e+04,eth,0.0,0.741262,56.029444,...,0.000188,0.000189,0.000179,0.000160,0.000302,0.000215,0.000205,0.000273,0.000209,0.000497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,2021-06-30 23:00:00,0.25375,0.25512,0.25192,0.25391,3.729461e+07,doge,0.0,4.220380,0.005068,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412
719,2021-06-30 23:00:00,2257.30000,2281.92000,2255.32000,2275.68000,3.543225e+04,eth,0.0,1.657382,40.438589,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412
719,2021-06-30 23:00:00,19.33300,19.57400,19.31800,19.51600,1.813782e+05,link,0.0,3.388434,0.387212,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412
719,2021-06-30 23:00:00,143.51000,144.62000,143.13000,144.26000,1.647400e+04,ltc,0.0,2.968342,2.205526,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2020-01-01 to 2020-12-31

In [46]:
train = data_split(df, '2020-02-02','2020-12-31')

## Environment for Portfolio Allocation


In [8]:
stock_dimension = len(config.ACTUAL_TICKERS)
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
features = config.TECHNICAL_INDICATORS_LIST + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]
features += [feature for feature in df.columns if feature.startswith("cov_")]

env_kwargs = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": train.tic.unique(),
    "all_tickers": train.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.01
    
}

e_train_gym = CustomTradingEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()

Stock Dimension: 8, State Space: 8


<a id='5'></a>
# Part 6: Implement DRL Algorithms

In [9]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [10]:
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS, tensorboard_log = config.TENSORBOARD_LOG_DIR)

Using cuda device


In [11]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=10000)

Logging to /home/joan/Desktop/TradingRLBot/tensorboard_log_from_main/a2c_3
------------------------------------
| time/                 |          |
|    fps                | 47       |
|    iterations         | 100      |
|    time_elapsed       | 10       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -11.4    |
|    explained_variance | -0.0811  |
|    learning_rate      | 0.0002   |
|    n_updates          | 99       |
|    policy_loss        | -1.42    |
|    std                | 1        |
|    value_loss         | 0.0239   |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 50       |
|    iterations         | 200      |
|    time_elapsed       | 19       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -11.4    |
|    explained_variance | -4.75    |
|    learning_rate      | 0.0002   |


### Model 2: **PPO**

In [12]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.005,
#     "learning_rate": 0.0001,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [13]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=80000)

## Trading
Assume that we have $1,000,000 initial capital at 2021-01-01.

In [14]:
trade = data_split(df,config.START_TEST_DATE, config.END_DATE)
e_trade_gym = CustomTradingEnv(df = trade, **env_kwargs)

In [15]:
trade.shape

(34752, 82)

In [16]:
trade

,date,open,high,low,close,volume,tic,psar,ui,atr,...,cov_4_4,cov_4_5,cov_4_6,cov_4_7,cov_5_5,cov_5_6,cov_5_7,cov_6_6,cov_6_7,cov_7_7
0,2021-01-01 00:00:00,0.181340,0.181460,0.178310,0.180510,1.919492e+07,ada,0.0,3.174316,0.003072,...,0.000069,0.000071,0.000066,0.000068,0.000204,0.000079,0.000089,0.000110,0.000091,0.000217
0,2021-01-01 00:00:00,37.359600,37.442300,36.963600,37.376400,9.511383e+04,bnb,1.0,2.328882,0.404216,...,0.000069,0.000071,0.000066,0.000068,0.000204,0.000079,0.000089,0.000110,0.000091,0.000217
0,2021-01-01 00:00:00,28923.630000,29031.340000,28690.170000,28995.130000,2.311811e+03,btc,0.0,1.505422,378.624288,...,0.000069,0.000071,0.000066,0.000068,0.000204,0.000079,0.000089,0.000110,0.000091,0.000217
0,2021-01-01 00:00:00,0.004672,0.004701,0.004601,0.004679,2.768207e+07,doge,0.0,2.016111,0.000074,...,0.000069,0.000071,0.000066,0.000068,0.000204,0.000079,0.000089,0.000110,0.000091,0.000217
0,2021-01-01 00:00:00,736.420000,739.000000,729.330000,734.070000,2.793270e+04,eth,0.0,1.998452,9.626878,...,0.000069,0.000071,0.000066,0.000068,0.000204,0.000079,0.000089,0.000110,0.000091,0.000217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4343,2021-06-30 23:00:00,0.253750,0.255120,0.251920,0.253910,3.729461e+07,doge,0.0,4.220380,0.005068,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412
4343,2021-06-30 23:00:00,2257.300000,2281.920000,2255.320000,2275.680000,3.543225e+04,eth,0.0,1.657382,40.438589,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412
4343,2021-06-30 23:00:00,19.333000,19.574000,19.318000,19.516000,1.813782e+05,link,0.0,3.388434,0.387212,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412
4343,2021-06-30 23:00:00,143.510000,144.620000,143.130000,144.260000,1.647400e+04,ltc,0.0,2.968342,2.205526,...,0.000200,0.000205,0.000185,0.000164,0.000317,0.000227,0.000209,0.000266,0.000202,0.000412


In [17]:
allocations, transactions, allocation_values = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

hit end!


In [18]:
allocations

,cash,ada,bnb,btc,doge,eth,link,ltc,xrp,date
0,997120.239953,29.578689,76.141852,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2021-01-01 01:00:00
0,15343.588965,0.000000,144.968534,32.919061,100.000000,0.0,0.000000,10.044596,0.000000,2021-01-01 02:00:00
0,956748.694060,62.074769,244.968534,0.000000,200.000000,0.0,100.000000,47.393274,0.000000,2021-01-01 03:00:00
0,864451.353531,68.152926,344.968534,0.000000,300.000000,100.0,94.801308,147.393274,0.000000,2021-01-01 04:00:00
0,37.702234,35.387087,434.547800,29.000000,336.425906,106.0,31.852700,157.393274,100.000000,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...
0,121.676146,350.559589,7807.196773,0.000000,174.197809,0.0,0.000000,90.437839,165.835100,2021-06-30 17:00:00
0,5793.040705,398.258921,7807.196773,0.000000,274.197809,0.0,1.000000,48.570896,155.416027,2021-06-30 18:00:00
0,1695.822245,346.670004,7826.196773,0.000000,174.197809,0.0,14.000000,36.241929,199.017638,2021-06-30 19:00:00
0,2032.904908,286.508366,7831.196773,0.000000,103.979163,0.0,29.000000,21.821604,187.141694,2021-06-30 20:00:00


In [19]:
transactions

,date,operation,tic,amount,price
0,2021-01-01 00:00:00,buy,ada,29.578689,0.18051
0,2021-01-01 00:00:00,buy,bnb,76.141852,37.37640
0,2021-01-01 01:00:00,sell,ada,29.578689,0.18358
0,2021-01-01 01:00:00,buy,bnb,68.826681,37.61340
0,2021-01-01 01:00:00,buy,btc,32.919061,29409.99000
...,...,...,...,...,...
0,2021-06-30 20:00:00,buy,bnb,7.000000,296.81000
0,2021-06-30 20:00:00,sell,doge,53.605026,0.25140
0,2021-06-30 20:00:00,buy,link,1.973107,19.22900
0,2021-06-30 20:00:00,sell,ltc,21.821604,142.16000


In [20]:
allocation_values

,cash,ada,bnb,btc,doge,eth,link,ltc,xrp,date
0,997120.239953,5.430056,2.863954e+03,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2021-01-01 01:00:00
0,15343.588965,0.000000,5.503006e+03,961060.469593,0.475510,0.0,0.000000,1339.245931,0.000000,2021-01-01 02:00:00
0,956748.694060,11.354717,9.290432e+03,0.000000,0.995620,0.0,1150.060000,6239.798495,0.000000,2021-01-01 03:00:00
0,864451.353531,12.390202,1.306403e+04,0.000000,1.468140,74229.0,1086.185989,19289.357809,0.000000,2021-01-01 04:00:00
0,37.702234,6.474775,1.638806e+04,846423.290000,1.662583,78508.9,364.493629,20599.631741,23.819000,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...
0,121.676146,467.891883,2.256046e+06,0.000000,43.335189,0.0,0.000000,12596.182253,110.479344,2021-06-30 17:00:00
0,5793.040705,541.313526,2.290085e+06,0.000000,69.210269,0.0,19.138000,6838.296452,105.092317,2021-06-30 18:00:00
0,1695.822245,481.628637,2.325241e+06,0.000000,44.035464,0.0,269.514000,5130.044990,136.028556,2021-06-30 19:00:00
0,2032.904908,392.602414,2.324378e+06,0.000000,26.140362,0.0,557.641000,3102.159227,128.323060,2021-06-30 20:00:00


<a id='6'></a>
# Part 7: Backtest

In [21]:
%load_ext autoreload
%autoreload 
bat = BackTest(trained_a2c, e_trade_gym)
bat.create_summary(allocations, allocation_values, transactions)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Start date,2021-01-01
End date,2021-06-30
Total months,8
,Backtest
Annual return,291.2%
Cumulative returns,166.375%
Annual volatility,162.4%
Sharpe ratio,1.59
Calmar ratio,4.48
Stability,0.55
Max drawdown,-65.029%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,65.03,2021-05-03,2021-06-22,NaT,NaN
1,51.96,2021-01-01,2021-01-21,2021-02-09,28
2,37.35,2021-02-19,2021-02-28,2021-04-05,32
3,19.85,2021-04-12,2021-04-18,2021-04-30,15
4,6.95,2021-04-06,2021-04-07,2021-04-08,3


Stress Events,mean,min,max
New Normal,1.03%,-34.04%,67.91%


Top 10 long positions of all time,max
bnb,99.80%
doge,19.81%
ltc,17.52%
ada,9.50%
xrp,6.33%
btc,2.18%
link,1.50%
eth,1.28%


Top 10 short positions of all time,max


Top 10 positions of all time,max
bnb,99.80%
doge,19.81%
ltc,17.52%
ada,9.50%
xrp,6.33%
btc,2.18%
link,1.50%
eth,1.28%
index,0.00%


In [22]:
bat.plot_return_against_hold(allocation_values)

In [23]:
%load_ext autoreload
%autoreload 
from trade.time_series_validation import TimeSeriesValidation
tsv = TimeSeriesValidation(num_splits=5, total_timesteps_model=1000, with_graphs=False)

model_name = "a2c"
model_params = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}

stock_dimension = len(df.tic.unique())
state_space = stock_dimension
features = config.TECHNICAL_INDICATORS_LIST + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]
features += [feature for feature in df.columns if feature.startswith("cov_")]

env_params = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": df.tic.unique(),
    "all_tickers": df.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.01
    
}


results = tsv.run(df, env_params, model_name, model_params)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train from [2020-06-29 00:00:00] to [2020-08-23 00:00:00]
Using cuda device
------------------------------------
| time/                 |          |
|    fps                | 61       |
|    iterations         | 100      |
|    time_elapsed       | 8        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -11.4    |
|    explained_variance | 0.499    |
|    learning_rate      | 0.0002   |
|    n_updates          | 99       |
|    policy_loss        | -3.83    |
|    std                | 1        |
|    value_loss         | 0.0892   |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 59       |
|    iterations         | 200      |
|    time_elapsed       | 16       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss     

In [57]:
results

{'return': -0.7290900184732636,
 'sharpe': -92.7592914916186,
 'sortino': -127.53311398552046,
 'max_drawdown': -0.7290400077273029}

In [64]:


f(a=2,a=4)

SyntaxError: keyword argument repeated: a (<ipython-input-64-70ae18d4ebeb>, line 4)

In [63]:
f"{ret:0.2}"

'-0.73'

In [25]:
with open("/home/joan/Desktop/TradingRLBot/hyperparameter_tuning/log/1625393799_ppo/trial_1_ppo.pkl", 'rb') as fp:
    data = pickle.load(fp)
data

FileNotFoundError: [Errno 2] No such file or directory: '/home/joan/Desktop/TradingRLBot/hyperparameter_tuning/log/1625393799_ppo/trial_1_ppo.pkl'